In [3]:
import sys
!{sys.executable} -m pip install wikipedia-api


In [3]:
import sys
!{sys.executable} -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 11.2 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 12.3 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 12.0 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.8 MB 11.6 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 10.6 MB/s eta 0:00:01
     ---------------------------------- ----- 11.0/12.8 MB 9.8 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 9.1 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.8 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import sys
!{sys.executable} -m pip install torch --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [5]:
import sys
!{sys.executable} -m pip install tf-keras

import tf_keras
print("tf_keras OK")


tf_keras OK


In [8]:
import sys
!{sys.executable} -m pip install -U sentencepiece protobuf


  Using cached protobuf-6.33.4-cp310-abi3-win_amd64.whl.metadata (593 bytes)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 12.8 MB/s  0:00:00
Using cached protobuf-6.33.4-cp310-abi3-win_amd64.whl (436 kB)

   ---------------------------------------- 0/2 [sentencepiece]
  Attempting uninstall: protobuf
   ---------------------------------------- 0/2 [sentencepiece]
    Found existing installation: protobuf 5.29.4
   ---------------------------------------- 0/2 [sentencepiece]
   -------------------- ------------------- 1/2 [protobuf]
    Uninstalling protobuf-5.29.4:
   -------------------- ------------------- 1/2 [protobuf]
      Successfully uninstalled protobuf-5.29.4
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   ----

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlepaddle-gpu 2.6.2 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 6.33.4 which is incompatible.


In [6]:
import sys
!{sys.executable} -m pip install torch --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# main.py
"""
🚀 MAIN SCRIPT - Pipeline Complet Veille NLP
"""

import sys
import io
import json
import logging
from pathlib import Path
from datetime import datetime

# ──────────────────────────────────────────────────────────────────────────────
# Forcer UTF-8 (OK terminal Windows, ignore si Jupyter)
# ──────────────────────────────────────────────────────────────────────────────
for _stream_name in ("stdout", "stderr"):
    stream = getattr(sys, _stream_name)
    if hasattr(stream, "reconfigure"):
        stream.reconfigure(encoding="utf-8", errors="replace")
    elif hasattr(stream, "buffer"):
        setattr(
            sys,
            _stream_name,
            io.TextIOWrapper(stream.buffer, encoding="utf-8", errors="replace"),
        )
    else:
        # Jupyter: pas de buffer -> on laisse tel quel
        pass

# ──────────────────────────────────────────────────────────────────────────────
# Ajouter src au path (compatible script + notebook)
# ──────────────────────────────────────────────────────────────────────────────
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    # Notebook / interactive (pas de __file__)
    BASE_DIR = Path.cwd()

SRC_DIR = BASE_DIR / "src"
sys.path.insert(0, str(SRC_DIR))

from news_collector import NewsCollector
from text_preprocessor import TextPreprocessor
from news_classifier import NewsClassifier
from report_generator import ReportGenerator

# ──────────────────────────────────────────────────────────────────────────────
# SETUP LOGGING
# ──────────────────────────────────────────────────────────────────────────────
def setup_logging(config: dict):
    """Configurer logging"""
    log_level = config.get("logging", {}).get("level", "INFO")
    log_file = config.get("logging", {}).get("log_file", "veille_system.log")

    logging.basicConfig(
        level=getattr(logging, log_level, logging.INFO),
        format="%(asctime)s | %(name)s | %(levelname)s | %(message)s",
        handlers=[
            logging.FileHandler(log_file, encoding="utf-8"),
            logging.StreamHandler(sys.stdout),
        ],
    )

    logger = logging.getLogger(__name__)
    logger.info("🚀 Pipeline Veille NLP - Démarrage")
    logger.info("Configuration chargée depuis: config.json")
    return logger

# ──────────────────────────────────────────────────────────────────────────────
# ÉTAPE 1 : COLLECTE
# ──────────────────────────────────────────────────────────────────────────────
def step_collect(config: dict, logger) -> list:
    logger.info("\n" + "=" * 75)
    logger.info("ÉTAPE 1 : COLLECTE ARTICLES")
    logger.info("=" * 75)

    collector = NewsCollector(config)
    articles = collector.collect_all()

    collector.save_to_jsonl("data/articles_raw.jsonl")
    collector.save_errors_log("data/collection_errors.json")

    logger.info(f"✅ ÉTAPE 1 COMPLÉTÉE : {len(articles)} articles collectés")
    return articles

# ──────────────────────────────────────────────────────────────────────────────
# ÉTAPE 2 : PRÉTRAITEMENT
# ──────────────────────────────────────────────────────────────────────────────
def step_preprocess(articles: list, config: dict, logger) -> list:
    logger.info("\n" + "=" * 75)
    logger.info("ÉTAPE 2 : PRÉTRAITEMENT NLP")
    logger.info("=" * 75)

    preprocessor = TextPreprocessor(config)
    processed_articles = preprocessor.process_batch(articles)

    preprocessor.print_quality_report()

    with open("data/articles_processed.jsonl", "w", encoding="utf-8") as f:
        for article in processed_articles:
            f.write(json.dumps(article, ensure_ascii=False) + "\n")

    logger.info(f"✅ ÉTAPE 2 COMPLÉTÉE : {len(processed_articles)} articles traités")
    return processed_articles

# ──────────────────────────────────────────────────────────────────────────────
# ÉTAPE 3 : CLASSIFICATION
# ──────────────────────────────────────────────────────────────────────────────
def step_classify(articles: list, config: dict, logger) -> list:
    logger.info("\n" + "=" * 75)
    logger.info("ÉTAPE 3 : CLASSIFICATION & EXTRACTION")
    logger.info("=" * 75)

    classifier = NewsClassifier(config)
    classified_articles = classifier.classify_batch(articles)

    classifier.print_classification_summary(classified_articles)

    with open("data/articles_classified.jsonl", "w", encoding="utf-8") as f:
        for article in classified_articles:
            f.write(json.dumps(article, ensure_ascii=False) + "\n")

    logger.info(f"✅ ÉTAPE 3 COMPLÉTÉE : {len(classified_articles)} articles classifiés")
    return classified_articles

# ──────────────────────────────────────────────────────────────────────────────
# ÉTAPE 4 : GÉNÉRATION RAPPORT
# ──────────────────────────────────────────────────────────────────────────────
def step_generate_report(articles: list, config: dict, logger) -> str:
    logger.info("\n" + "=" * 75)
    logger.info("ÉTAPE 4 : GÉNÉRATION RAPPORT")
    logger.info("=" * 75)

    generator = ReportGenerator(config)
    report = generator.generate(articles)

    output_file = config.get("output", {}).get("report_name", "veille_report.txt")
    output_dir = config.get("output", {}).get("output_dir", "./output")

    Path(output_dir).mkdir(parents=True, exist_ok=True)
    output_path = Path(output_dir) / output_file

    generator.save_report(report, str(output_path))
    logger.info("✅ ÉTAPE 4 COMPLÉTÉE : Rapport sauvegardé")

    return report

# ──────────────────────────────────────────────────────────────────────────────
# MAIN
# ──────────────────────────────────────────────────────────────────────────────
def main():
    # Créer dossiers
    Path("data").mkdir(exist_ok=True)
    Path("output").mkdir(exist_ok=True)

    config_path = BASE_DIR / "config.json"

    try:
        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)
    except FileNotFoundError:
        print("❌ ERREUR : config.json non trouvé")
        print(f"Chemin attendu : {config_path}")
        sys.exit(1)

    logger = setup_logging(config)

    try:
        articles = step_collect(config, logger)
        if not articles:
            logger.warning("⚠️ Aucun article collecté")
            return

        processed_articles = step_preprocess(articles, config, logger)
        if not processed_articles:
            logger.error("❌ Aucun article après prétraitement")
            return

        classified_articles = step_classify(processed_articles, config, logger)
        if not classified_articles:
            logger.error("❌ Aucun article après classification")
            return

        report = step_generate_report(classified_articles, config, logger)

        logger.info("\n" + "=" * 75)
        logger.info("✅ PIPELINE COMPLET - SUCCÈS")
        logger.info("=" * 75)
        logger.info(f"Rapport généré: output/{config.get('output', {}).get('report_name', 'veille_report.txt')}")
        logger.info(f"Articles traités: {len(classified_articles)}")
        logger.info(f"Fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        logger.info("=" * 75)

        print("\n" + "=" * 75)
        print("📄 APERÇU RAPPORT")
        print("=" * 75)
        print("\n".join(report.split("\n")[:30]))
        print("...")
        print(f"\n✅ Rapport complet sauvegardé en output/{config.get('output', {}).get('report_name', 'veille_report.txt')}")

    except Exception as e:
        logger.error(f"\n❌ ERREUR FATALE : {str(e)}")
        logger.exception("Traceback:")
        sys.exit(1)

if __name__ == "__main__":
    main()


2026-01-20 16:47:24,312 | __main__ | INFO | 🚀 Pipeline Veille NLP - Démarrage
2026-01-20 16:47:24,313 | __main__ | INFO | Configuration chargée depuis: config.json
2026-01-20 16:47:24,314 | __main__ | INFO | 
2026-01-20 16:47:24,315 | __main__ | INFO | ÉTAPE 1 : COLLECTE ARTICLES
2026-01-20 16:47:24,316 | __main__ | INFO | ===========================================================================
2026-01-20 16:47:24,320 | news_collector | INFO | ======================================================================
2026-01-20 16:47:24,321 | news_collector | INFO | 📥 COLLECTE ARTICLES - Début
2026-01-20 16:47:24,322 | news_collector | INFO | ======================================================================
2026-01-20 16:47:24,323 | news_collector | INFO | 🔄 Collecte HackerNews...
2026-01-20 16:47:24,325 | news_collector | INFO |   Scraping page 1/10


2026-01-20 16:47:27,097 | news_collector | INFO |   Scraping page 2/10
2026-01-20 16:47:29,322 | news_collector | INFO |   Scraping page 3/10
2026-01-20 16:47:31,540 | news_collector | INFO |   Scraping page 4/10
2026-01-20 16:47:33,762 | news_collector | INFO |   Scraping page 5/10
2026-01-20 16:47:35,996 | news_collector | INFO |   Scraping page 6/10
2026-01-20 16:47:38,213 | news_collector | INFO |   Scraping page 7/10
2026-01-20 16:47:40,434 | news_collector | INFO |   Scraping page 8/10
2026-01-20 16:47:42,653 | news_collector | INFO |   Scraping page 9/10
2026-01-20 16:47:44,873 | news_collector | INFO |   Scraping page 10/10
2026-01-20 16:47:47,092 | news_collector | INFO | ✅ HackerNews: 300 articles collectés
2026-01-20 16:47:47,092 | news_collector | INFO | 🔄 Collecte RSS: https://pycoders.com/feed
2026-01-20 16:47:47,606 | news_collector | INFO | ✅ RSS: 3 articles collectés
2026-01-20 16:47:47,606 | news_collector | INFO | 🔄 Collecte Wikipedia...
2026-01-20 16:47:47,615 | wik

Device set to use cpu


2026-01-20 16:47:57,814 | news_classifier | INFO | ✅ Modèle chargé: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli


Device set to use cpu


2026-01-20 16:48:04,378 | news_classifier | INFO | ✅ Sentiment classifier chargé
2026-01-20 16:48:04,379 | news_classifier | INFO | 🤖 Classification 307 articles...
2026-01-20 16:48:29,770 | news_classifier | INFO |   ✓ 10/307 articles
2026-01-20 16:49:01,213 | news_classifier | INFO |   ✓ 20/307 articles
2026-01-20 16:49:29,367 | news_classifier | INFO |   ✓ 30/307 articles
2026-01-20 16:49:55,895 | news_classifier | INFO |   ✓ 40/307 articles
2026-01-20 16:50:22,931 | news_classifier | INFO |   ✓ 50/307 articles
2026-01-20 16:50:49,893 | news_classifier | INFO |   ✓ 60/307 articles
2026-01-20 16:51:17,918 | news_classifier | INFO |   ✓ 70/307 articles
2026-01-20 16:51:43,398 | news_classifier | INFO |   ✓ 80/307 articles
2026-01-20 16:52:08,511 | news_classifier | INFO |   ✓ 90/307 articles
2026-01-20 16:52:35,424 | news_classifier | INFO |   ✓ 100/307 articles
2026-01-20 16:53:03,467 | news_classifier | INFO |   ✓ 110/307 articles
2026-01-20 16:53:32,030 | news_classifier | INFO |  